In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
import datetime

import warnings

# Suppress pandas-specific warnings
warnings.filterwarnings("ignore")

In [2]:
years_cal = ['2019','2020','2021','2022','2023','2024']
yearly_metric = {}
for year in years_cal:
    yearly_metric[year] = pd.read_excel('yearly_metric.xlsx', sheet_name=year, index_col=0)

In [16]:
inside_list = ['HIEP.NS', 'NAM.TH.CS', 'NAM.TH.PS','MANDATE VND', 'VNI']

# So sánh performance hàng năm

In [45]:
for key in yearly_metric.keys():
    fig = go.Figure()
    
    # top 3 except us:
    top_3 = yearly_metric[key][yearly_metric[key].index.isin(inside_list) == False].sort_values(by = 'Annual Return', ascending = False)[:3].index
    for col in yearly_metric[key].index:
        if col == 'NAM.TH.PS':
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='red', size=30))) # Set color to red and increase thickness
        elif col == 'NAM.TH.CS':
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='green', size=30))) # Set color to red and increase thickness
        elif col == 'HIEP.NS':
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='blue', size=30))) # Set color to red and increase thickness
        elif col == 'MANDATE VND':
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='orange', size=30))) # Set color to red and increase thickness
        elif col == 'VNI':
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='purple', size=25, symbol = "square"))) # Set color to red and increase thickness
        elif col in top_3:
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='maroon', size = 15, symbol = "diamond"))) # Set color to blur gray and reduce thickness
        else:
            fig.add_trace(go.Scatter(x=[yearly_metric[key].loc[col]['Annual Return']], y=[yearly_metric[key].loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='bisque', size = 5))) # Set color to blur gray and reduce thickness

    fig.update_layout(title='Compare Annual Return / Max Drawdown {}'.format(key),
                      xaxis_title='Annual Return',
                      yaxis_title='Max Drawdown')

    fig.show()

In [4]:
annual_ret = pd.DataFrame(index = yearly_metric['2024'].index)
for key in yearly_metric.keys():
    annual_ret = pd.concat([annual_ret, yearly_metric[key]['Annual Return']], axis = 1)

annual_ret.columns = yearly_metric.keys()

In [5]:
max_drawdown = pd.DataFrame(index = yearly_metric['2024'].index)
for key in yearly_metric.keys():
    max_drawdown = pd.concat([max_drawdown, yearly_metric[key]['Max Drawdown']], axis = 1)

max_drawdown.columns = yearly_metric.keys()

# Đánh giá performance 6 năm

In [6]:
cagr_6y = []
max_dd_6y = []
for tick in annual_ret.replace(0.000, np.nan).dropna().index:
    cagr = (1 + annual_ret.loc[tick]).cumprod().iloc[-1] ** (1/6) - 1
    max_dd_6y.append(max_drawdown.loc[tick].min())
    cagr_6y.append(cagr)

performance_6Y = pd.DataFrame({
    'CAGR': cagr_6y,
    'Max Drawdown': max_dd_6y
}, index = annual_ret.replace(0.000, np.nan).dropna().index )

In [44]:
fig = go.Figure()
top_3_6Y = performance_6Y[performance_6Y.index.isin(inside_list) == False].sort_values(by = 'CAGR', ascending = False)[:3].index

for col in performance_6Y.index:
    if col == 'HIEP.NS':
        fig.add_trace(go.Scatter(x=[performance_6Y.loc[col]['CAGR']], y=[performance_6Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='blue', size=30))) # Set color to red and increase thickness
    elif col == 'MANDATE VND':
        fig.add_trace(go.Scatter(x=[performance_6Y.loc[col]['CAGR']], y=[performance_6Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='orange', size=30))) # Set color to red and increase thickness
    elif col == 'VNI':
            fig.add_trace(go.Scatter(x=[performance_6Y.loc[col]['CAGR']], y=[performance_6Y.loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='purple', size=25, symbol = "square"))) # Set color to red and increase thickness
    elif col in top_3_6Y:
            fig.add_trace(go.Scatter(x=[performance_6Y.loc[col]['CAGR']], y=[performance_6Y.loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='maroon', size = 15, symbol = "diamond"))) # Set color to blur gray and reduce thickness
    else:
        fig.add_trace(go.Scatter(x=[performance_6Y.loc[col]['CAGR']], y=[performance_6Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='lightgray', size = 10))) # Set color to blur gray and reduce thickness

fig.update_layout(title='Compare CAGR / Max Drawdown 2019 - 2024',
                  xaxis_title='CAGR',
                  yaxis_title='Max Drawdown')

fig.show()

# Đánh giá performance 3 năm

In [8]:
cagr_3y = []
max_dd_3y = []
for tick in annual_ret.iloc[:,3:].replace(0.000, np.nan).dropna().index:
    cagr = (1 + annual_ret.iloc[:,3:].loc[tick]).cumprod().iloc[-1] ** (1/3) - 1
    max_dd_3y.append(max_drawdown.iloc[:,3:].loc[tick].min())
    cagr_3y.append(cagr)

performance_3Y = pd.DataFrame({
    'CAGR': cagr_3y,
    'Max Drawdown': max_dd_3y
}, index = annual_ret.iloc[:,3:].replace(0.000, np.nan).dropna().index )

In [43]:
fig = go.Figure()
top_3_3Y = performance_3Y[performance_3Y.index.isin(inside_list) == False].sort_values(by = 'CAGR', ascending = False)[:3].index

for col in performance_3Y.index:
    if col == 'NAM.TH.PS':
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='red', size=30))) # Set color to red and increase thickness
    elif col == 'NAM.TH.CS':
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='green', size=30))) # Set color to red and increase thickness
    elif col == 'HIEP.NS':
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='blue', size=30))) # Set color to red and increase thickness
    elif col == 'MANDATE VND':
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='orange', size=30))) # Set color to red and increase thickness
    elif col == 'VNI':
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='purple', size=25, symbol = "square"))) # Set color to red and increase thickness
    elif col in top_3_3Y:
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                             mode='markers', name=col, marker=dict(color='maroon', size = 15, symbol = "diamond"))) # Set color to blur gray and reduce thickness
    else:
        fig.add_trace(go.Scatter(x=[performance_3Y.loc[col]['CAGR']], y=[performance_3Y.loc[col]['Max Drawdown']],
                         mode='markers', name=col, marker=dict(color='lightgray', size = 10))) # Set color to blur gray and reduce thickness

fig.update_layout(title='Compare CAGR / Max Drawdown 2022 - 2024',
                  xaxis_title='Annual Return',
                  yaxis_title='Max Drawdown')

fig.show()